In [1]:
# Importamos las librerías necesarias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
import pickle
from sklearn.cluster import KMeans
from random import randrange

# 1. ENTRENAR MODELO

In [2]:
#Cargarmos los datos
movies = pd.read_excel('data/movies_min.xlsx', index_col=0)
#Dimensiones del dataset
print("Nuestro dataset tiene [%s] filas y [%s] columnas" % (movies.shape[0], movies.shape[1]))

Nuestro dataset tiene [671] filas y [99] columnas


In [3]:
# Entrenamos el modelo para que agrupe los datos en 12 clusters
kmeans = KMeans(n_clusters=12, algorithm='full', random_state=2).fit(movies.fillna(0))

In [4]:
# Le pasamos todo el dataset a nuestro modelo, para que nos diga a que grupo pertenece cada usuario.
predictions  = kmeans.predict(movies.fillna(0))
clustered_users = pd.concat([movies.reset_index(), pd.DataFrame({'group':predictions})], axis=1)
#Imprimimos el dataset con una nueva columna al final que nos indica el grupo al que pertenece cada usuario. 
n_usuario = 3
print("Ejemplo: El usuario [%s] pertenece al grupo/cluster [%s]" % (clustered_users.iloc[n_usuario,0], clustered_users['group'][n_usuario]))
clustered_users

Ejemplo: El usuario [3] pertenece al grupo/cluster [8]


,user,Forrest Gump (1994),Pulp Fiction (1994),"Shawshank Redemption, The (1994)","Silence of the Lambs, The (1991)",Star Wars: Episode IV - A New Hope (1977),Jurassic Park (1993),"Matrix, The (1999)",Toy Story (1995),Schindler's List (1993),...,2001: A Space Odyssey (1968),There's Something About Mary (1998),Finding Nemo (2003),Four Weddings and a Funeral (1994),GoldenEye (1995),"Clockwork Orange, A (1971)","Dark Knight, The (2008)",Shakespeare in Love (1998),Clueless (1995),group
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
1,1,3.0,4.0,NaN,3.0,NaN,4.0,NaN,NaN,4.0,...,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,5.0,5
2,2,5.0,4.5,5.0,3.0,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,3.0,NaN,NaN,NaN,3.0,NaN,NaN,7
3,3,5.0,5.0,NaN,NaN,5.0,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,4.0,5.0,NaN,NaN,NaN,8
4,4,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.5,4.0,NaN,NaN,NaN,NaN,NaN,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,666,4.0,5.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,10
667,667,NaN,5.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
668,668,NaN,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4
669,669,NaN,NaN,5.0,5.0,NaN,NaN,4.0,4.0,5.0,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7


In [5]:
# Ya tenemos nuestro modelo entrenado, ahora queremos guardarlo para poder usarlo en nuestra app web.
filename = 'kmeans.sav'
pickle.dump(kmeans, open(filename, 'wb'))

# 2. CLUSTERIZAR NUEVOS DATOS

Ya tenemos el modelo entrenado. Hemos aplicado nuestro modelo a los datos que teníamos y nos ha agrupado los usuarios.
Ahora queremos predecir a qué grupo pertenecerá un nuevo usuario, a partir de sus votaciones.

## 2.1. Inicializar modelo

In [53]:
def inicializar(movies):
     #Cargar modelo y clusterizar usuarios
    kmeans = pickle.load(open('kmeans.sav', 'rb'))
    predictions  = kmeans.predict(movies.fillna(0)) # Esto nos devuelve, para cada usuario, el grupo al que pertenece
    #Concatenar el grupo de cada usuario al dataset inicial.
    clustered_users = pd.concat([movies.reset_index(), pd.DataFrame({'group':predictions})], axis=1)
    return clustered_users 
clustered_users = inicializar(movies)

## 2.2. Seleccionar peliculas para que las vote el usuario
Con esta funcion obtenemos una lista de 'n' películas al azar.
¿Qué hacemos con esto? Pasar la lista a la vista HTML e imprimir los nombres de las pelis, para que el usuario las vote.

In [6]:
def selectRandomMovies(movies, n):
    max_pelis = movies.shape[1]
    lista_pelis = []
    for i in range(n):
        random_column = randrange(max_pelis) # num aleatorio entre 0 y max_pelis
        lista_pelis.append(movies.iloc[:,random_column].name) # seleccionar columna
    return lista_pelis

In [7]:
lista_pelis = selectRandomMovies(movies, 10)
for peli in lista_pelis:
    print(peli)

Ocean's Eleven (2001)
Star Wars: Episode IV - A New Hope (1977)
Lord of the Rings: The Fellowship of the Ring, The (2001)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
Dumb & Dumber (Dumb and Dumber) (1994)
Fugitive, The (1993)
Apollo 13 (1995)
Die Hard: With a Vengeance (1995)
One Flew Over the Cuckoo's Nest (1975)
Memento (2000)


## 2.3. Obtener las votaciones del usuario
Tenemos que recuperar, desde la vista HTML (mediante un formulario) las votaciones de las peliculas.
Las votaciones tenemos que almacenarlas en una lista, como esta:

In [49]:
lista_votaciones = [1,1,5,5,5,1,1,1,4,5]
votaciones_usuario = pd.DataFrame(columns = movies.columns).append(pd.Series(name='ToPredict'))
# Añadir votaciones a la votación vacía
for i in range(len(lista_pelis)):
    votaciones_usuario.loc['ToPredict'][lista_pelis[i]] = lista_votaciones[i]

'''
los valores vacios que quedan (pelis que no ha votado/no ha visto) las rellenamos con
el valor medio de las posibles votaciones -> 2.5, para que no impacte demasiado al hacer el predict
'''
votaciones_usuario = votaciones_usuario.fillna(2.5)
votaciones_usuario

,Forrest Gump (1994),Pulp Fiction (1994),"Shawshank Redemption, The (1994)","Silence of the Lambs, The (1991)",Star Wars: Episode IV - A New Hope (1977),Jurassic Park (1993),"Matrix, The (1999)",Toy Story (1995),Schindler's List (1993),Terminator 2: Judgment Day (1991),...,Trainspotting (1996),2001: A Space Odyssey (1968),There's Something About Mary (1998),Finding Nemo (2003),Four Weddings and a Funeral (1994),GoldenEye (1995),"Clockwork Orange, A (1971)","Dark Knight, The (2008)",Shakespeare in Love (1998),Clueless (1995)
ToPredict,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,...,2.5,2.5,2.5,1,2.5,2.5,2.5,5,2.5,2.5


## 2.4. Obtener recomendaciones para el usuario

In [51]:
'''
hacemos el predict para que nos devuelva un grupo de usuarios al que pertenece nuestro usuario
'''
predicted_group = kmeans.predict(votaciones_usuario.iloc[0:1])
# de todos los clusters, filtramos para quedarnos con el cluster de nuestro usuario
cluster = clustered_users[clustered_users['group']==int(predicted_group)]
#filtramos el cluster y nos quedamos solo con las pelis que no ha visto nuestro usuario
pelis_no_vistas = cluster.loc[:, ~cluster.columns.isin(lista_pelis)]
print(pelis_no_vistas.drop(['user','group'], axis=1).keys())

Index(['Forrest Gump (1994)', 'Pulp Fiction (1994)',
       'Shawshank Redemption, The (1994)', 'Silence of the Lambs, The (1991)',
       'Star Wars: Episode IV - A New Hope (1977)', 'Jurassic Park (1993)',
       'Matrix, The (1999)', 'Toy Story (1995)', 'Schindler's List (1993)',
       'Terminator 2: Judgment Day (1991)',
       'Star Wars: Episode V - The Empire Strikes Back (1980)',
       'Braveheart (1995)', 'Back to the Future (1985)', 'Fargo (1996)',
       'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
       'Independence Day (a.k.a. ID4) (1996)',
       'Star Wars: Episode VI - Return of the Jedi (1983)', 'Aladdin (1992)',
       'Fugitive, The (1993)', 'Dances with Wolves (1990)',
       'Seven (a.k.a. Se7en) (1995)', 'Lion King, The (1994)',
       'Godfather, The (1972)',
       'Lord of the Rings: The Fellowship of the Ring, The (2001)',
       'Apollo 13 (1995)', 'True Lies (1994)',
       'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',